In [22]:
data_root <- "/shared/0/projects/news-quotes/"

In [23]:
library("lme4")
library("margins")
library("stargazer")
library("ggeffects")
library("broom")

In [24]:
# help(glmer)

In [25]:
mydata <- read.csv(paste(data_root, "reg_data.csv", sep = ''), header = TRUE)

In [26]:
nrow(mydata)

[1] 524052

In [27]:
ncol(mydata)

[1] 249

In [28]:
# coarsen reporters into 4 groups.
mydata$reporter_eth_ethnea_broad <- as.character(mydata$reporter_eth_ethnea_broad)
mydata$reporter_eth_ethnea_broad[mydata$reporter_eth_ethnea_broad == 'EasternEuropean'] <- 'European'
mydata$reporter_eth_ethnea_broad[mydata$reporter_eth_ethnea_broad == 'WesternNorthernEuropean'] <- 'European'
mydata$reporter_eth_ethnea_broad[mydata$reporter_eth_ethnea_broad == 'SouthernEuropean'] <- 'European'
mydata$reporter_eth_ethnea_broad[mydata$reporter_eth_ethnea_broad == 'Chinese'] <- 'Asian'
mydata$reporter_eth_ethnea_broad[mydata$reporter_eth_ethnea_broad == 'Indian'] <- 'Asian'
mydata$reporter_eth_ethnea_broad[mydata$reporter_eth_ethnea_broad == 'EastAsian'] <- 'Asian'
mydata$reporter_eth_ethnea_broad[mydata$reporter_eth_ethnea_broad == 'MiddleEastern'] <- 'OtherUnknown'
mydata$reporter_eth_ethnea_broad[mydata$reporter_eth_ethnea_broad == 'African'] <- 'OtherUnknown'
mydata$reporter_eth_ethnea_broad[mydata$reporter_eth_ethnea_broad == 'unknown'] <- 'OtherUnknown'
mydata$reporter_eth_ethnea_broad <- as.factor(mydata$reporter_eth_ethnea_broad)

In [29]:
mydata <- within(mydata, author_eth_ethnea_broad <- relevel(author_eth_ethnea_broad, ref = 'English'))
mydata <- within(mydata, reporter_eth_ethnea_broad <- relevel(reporter_eth_ethnea_broad, ref = 'English'))
mydata <- within(mydata, author_gender_ethnea <- relevel(author_gender_ethnea, ref = 'M'))
mydata <- within(mydata, reporter_gender_ethnea <- relevel(reporter_gender_ethnea, ref = 'M'))

In [30]:
mydata <- within(mydata, author_pos_cate <- relevel(author_pos_cate, ref = 'last_position'))
mydata <- within(mydata, is_top_author <- relevel(is_top_author, ref = 'yes'))
mydata <- within(mydata, is_corresponding <- relevel(is_corresponding, ref = 'yes'))
mydata <- within(mydata, affiliation_cate <- relevel(affiliation_cate, ref = 'domestic'))

## Full data

In [31]:
base_str <- "is_author_mentioned ~ 1 + author_eth_ethnea_broad + author_gender_ethnea + reporter_eth_ethnea_broad + reporter_gender_ethnea + \
          last_name_length + last_name_prob + author_pos_cate + author_rank + is_top_author + is_corresponding + \
          affiliation_rank + affiliation_cate + num_authors + mention_year_center + gap_in_years + \
          num_words + num_mentioned_papers + FleschReadingEase + sentences_per_paragraph + type_token_ratio"

# 533 keywords
# keywords <- "Composite_material + Chemistry + Chromatography + Botany + Surgery + Medicine + Cognitive_psychology + Affect__psychology_ + Aggression + Psychology + Psychiatry + Cell_biology + Transcriptome + Molecular_biology + Carcinogenesis + Biology + Human_sexuality + Physical_therapy + Testosterone + Psychotherapist + Prostate_cancer + Mood + Disease + Genetics + Genome + Randomized_controlled_trial + Quality_of_life + Comorbidity + Severity_of_illness + Diabetes_mellitus + Prospective_cohort_study + Ideology + China + Law + Sociology + Environmental_engineering + Environmental_resource_management + Economics + Climate_change + Population + Evolutionary_biology + Cell + Phylogenetics + Ecology + Taxon + Biodiversity + Atmospheric_sciences + Environmental_science + Global_warming + Meteorology + Pedagogy + Social_science + Social_psychology + Confidence_interval + Referral + Young_adult + Medical_prescription + Molecule + Organic_chemistry + Materials_science + Environmental_health + Obesity + Body_mass_index + Public_health + Biochemistry + Endocrinology + Internal_medicine + Mitochondrion + Democracy + Political_economy + Public_administration + Politics + Public_opinion + Gerontology + Cohort_study + Lower_risk + Developmental_psychology + Paleontology + Geology + Neuroscience + Biophysics + RNA + Atomic_physics + Physics + Ion + Photon + Optics + Climatology + Geography + Precipitation + Chemical_engineering + Membrane + Inorganic_chemistry + Environmental_chemistry + Psychological_resilience + Risk_assessment + Cardiology + Cause_of_death + Atrial_fibrillation + Stimulus__physiology_ + Schizophrenia + Neuroimaging + Perception + Intensive_care_medicine + Nursing + Developing_country + Health_care + Drug + Distress + Political_science + Prefrontal_cortex + Social_relation + Chromatin + Microbiology + Antimicrobial + Antibiotics + Pregnancy + Pathology + Applied_psychology + Cognition + Anxiety + Radiology + Radiation_therapy + Biopsy + Chemotherapy + Multimedia + Autism + Socioeconomics + Agriculture + Gynecology + Breast_cancer + Obstetrics + Gestation + Pharmacology + Clinical_trial + Food_science + Escherichia_coli + Bacteria + Photochemistry + Injury_prevention + Human_factors_and_ergonomics + Suicide_prevention + Social_environment + Occupational_safety_and_health + Heart_failure + Predation + In_vivo + CRISPR + Crop + Carbon + Public_relations + Demography + Dentistry + Logistic_regression + Health_equity + Medicaid + Epidemiology + Threatened_species + Species_richness + Harm + Classical_mechanics + Quantum_mechanics + Odds_ratio + Homeostasis + Type_2_diabetes + Cohort + Anatomy + Interpersonal_relationship + Norm__social_ + Crystallography + Physiology + Placebo + MEDLINE + Pediatrics + Adverse_effect + Transplantation + Dopamine + Embryonic_stem_cell + Criminology + Astrophysics + Astronomy + Condensed_matter_physics + Optoelectronics + Molecular_physics + Nanotechnology + Crystal + Animal_science + Sediment + Melanoma + Cell_culture + Electronic_engineering + Odds + Overweight + Confounding + Communication + Child_development + Psychological_intervention + Gene + Management_science + Offspring + Epigenetics + Mental_health + Well_being + Immigration + Coping__psychology_ + Physical_exercise + Personality + Particle_physics + Alternative_medicine + Immunology + Big_Five_personality_traits + PsycINFO + Happiness + Extinction + Environmental_protection + Land_use + Agroforestry + Vegetation + Habitat + Ecosystem + Mineralogy + Geochemistry + Economic_growth + Vaccination + Recall + Working_memory + Radiation + Atmosphere + Vulnerability + Catalysis + Anesthesia + Toxicology + Cannabis + Government + European_union + Risk_factor + Systematic_review + General_surgery + Clinical_endpoint + Lung_cancer + Polymer + Geophysics + Geomorphology + Advertising + Cross_sectional_study + Interquartile_range + Weight_loss + Health_promotion + Academic_achievement + Finance + Chronic_pain + Immune_system + T_cell + Immunity + Virology + Dementia + Alzheimer_s_disease + Socioeconomic_status + Allele + Insulin + Hormone + Evidence_based_medicine + Meta_analysis + Medical_emergency + Zoology + Actuarial_science + Hydrology + Functional_magnetic_resonance_imaging + Electroencephalography + Machine_learning + Artificial_intelligence + Clinical_psychology + Nanoparticle + Laser + Ethnic_group + Cancer + Magnetic_field + Antigen + Antibody + Seismology + Addiction + Vitamin_D_and_neurology + Athletes + Marketing + Receptor + Social_support + Sleep_deprivation + Microeconomics + Legislation + Transcription_factor + Fertility + Dermatology + Pathogenesis + Apoptosis + Proinflammatory_cytokine + Ovarian_cancer + Stem_cell + Multivariate_analysis + Fishery + Mortality_rate + Virulence + Malaria + Knowledge_management + Analytical_chemistry + Graphene + Semiconductor + Coronary_artery_disease + Heart_disease + Cholesterol + Veterinary_medicine + Engineering + Biomarker__medicine_ + Electron + Microbiome + Gut_flora + Physical_medicine_and_rehabilitation + Stroke + Bioinformatics + Body_weight + Arctic + Poverty + Exoplanet + Planet + Stars + Foraging + National_Health_and_Nutrition_Examination_Survey + Urine + Hazard_ratio + Observational_study + Proportional_hazards_model + Inflammation + Kidney_disease + Gastroenterology + Text_mining + Locus__genetics_ + Genome_wide_association_study + Urology + Ranging + Survival_rate + Incentive + Phenomenon + Statistics + Longitudinal_study + Brain_mapping + Metabolic_syndrome + Agronomy + Asthma + Relative_risk + Breastfeeding + Endangered_species + Climate_model + Social_perception + Social_media + Social_network + Business + Etiology + Mesenchymal_stem_cell + Weight_gain + Mathematics + Nicotine + Emergency_department + Myocardial_infarction + Emergency_medicine + Labour_economics + Health_policy + Qualitative_research + Clinical_Practice + Guideline + Wildlife + Family_medicine + Regulation_of_gene_expression + Cellular_differentiation + microRNA + Downregulation_and_upregulation + Computer_science + Developed_country + Demographic_economics + Colorectal_cancer + Nutrient + Mutant + Cancer_research + Allergy + Biological_dispersal + Magnetic_resonance_imaging + Transmission__mechanics_ + Retrospective_cohort_study + Metastasis + Feeling + Metabolism + Signal_transduction + Traumatic_brain_injury + Genomics + DNA_methylation + Oncology + Mutation + Phenotype + Smoking_cessation + Black_hole + Air_pollution + Hippocampus + Biotechnology + Biomass + Volcano + Longevity + Empathy + Psychosocial + Greenhouse_gas + Focus_group + Regimen + Fetus + Computer_vision + Computational_biology + Gene_expression + DNA + Nuclear_magnetic_resonance + Solar_System + Astrobiology + Audiology + Circadian_rhythm + Rehabilitation + Toxicity + Global_health + Reproductive_health + Neurodegeneration + Galaxy + Virus + Innate_immune_system + Early_childhood + Amygdala + Vitamin + Adipose_tissue + Architecture + Data_mining + Quantum + Blood_pressure + Waste_management + Sustainability + Incidence__epidemiology_ + Substance_abuse + In_vitro + The_Internet + Attention_deficit_hyperactivity_disorder + Opioid + DNA_damage + Visual_perception + Sensory_system + Genotype + Antibiotic_resistance + Multiple_sclerosis + Case_control_study + Single_nucleotide_polymorphism + Cancer_cell + Trait + Empirical_research + Simulation + Oxidative_stress + Antioxidant + Progenitor_cell + Lung + Oceanography + Immunotherapy + Cytokine + Kinase + Development_economics + Cell_type + Social_cognition + Major_depressive_disorder + Hippocampal_formation + Central_nervous_system + Medical_record + Psychopathology + Skeletal_muscle + Transcription__biology_ + Ecosystem_services + Heart_rate + Outbreak + Phylogenetic_tree + Enzyme + Genetic_variation + Psychosis + Pathogen + History + Atom + Arousal + Remote_sensing + Crossover_study + Programmed_cell_death + Human_brain + Stimulation + Scattering + Antidepressant + Population_study + Corporate_governance + Interpersonal_communication + Osteoporosis + Alcohol + Biomedical_engineering + Induced_pluripotent_stem_cell + Demographics + Insulin_resistance + Autism_spectrum_disorder + Mindfulness + Limiting + Cretaceous + Spectroscopy + Prosocial_behavior + Computer_security + Gestational_age + Archaeology + Welfare + Mental_illness + Phosphorylation + Life_expectancy + Spin_half + Thin_film + Narrative + Gender_studies + Public_policy + Epilepsy + Metal + Instability + Particle + Spectral_line + Cell_growth + Cytotoxic_T_cell + Cycling + Intracellular + Ageing + Bipolar_disorder + Meal + Ingestion + DNA_sequencing + Amino_acid"

# 496 keywords
# keywords <- "Political_science + Melanoma + Dermatology + Pathology + Lower_risk + Risk_factor + Population + Medicine + Risk_assessment + Exoplanet + Astronomy + Astrophysics + Spin_half + Planet + Physics + Biochemistry + Enzyme + Molecule + Biology + Randomized_controlled_trial + Placebo + Medical_emergency + Drug + Asthma + Clinical_trial + Physical_therapy + Environmental_science + Biodiversity + Environmental_resource_management + Ecology + Genetics + Diabetes_mellitus + Immunology + Immune_system + Apoptosis + Microbiology + Immunity + Cytokine + Escherichia_coli + Nursing + Family_medicine + Cohort + Gerontology + Longevity + Health_care + Global_health + Body_mass_index + Odds + Cross_sectional_study + Logistic_regression + Confidence_interval + Public_health + Odds_ratio + Engineering + Development_economics + Government + Biophysics + Skeletal_muscle + Endocrinology + Internal_medicine + Climatology + Cancer_research + Cancer + In_vivo + Atomic_physics + Quantum_mechanics + Magnetic_field + Atom + Social_psychology + Mental_health + Substance_abuse + Clinical_psychology + Psychology + Surgery + Cell + Ranging + Galaxy + Stars + Anesthesia + Magnetic_resonance_imaging + Stroke + Chemistry + Chromatography + Meta_analysis + Young_adult + Pregnancy + Psychiatry + Observational_study + Clinical_Practice + Biopsy + Retrospective_cohort_study + Prospective_cohort_study + Developmental_psychology + Spectral_line + Social_relation + Public_relations + Wildlife + Business + Aggression + Psychosocial + Mindfulness + Psychological_intervention + Psychotherapist + Urology + Optoelectronics + Optics + Virology + Condensed_matter_physics + Classical_mechanics + Cohort_study + Qualitative_research + Quality_of_life + Phenomenon + Dementia + Cognition + Socioeconomic_status + Sustainability + Marketing + Disease + Pathogen + Social_network + Crop + Agronomy + Anatomy + Geology + Nicotine + Smoking_cessation + Organic_chemistry + Toxicity + Pharmacology + Gynecology + Heart_disease + Health_promotion + Alternative_medicine + Epidemiology + Overweight + Weight_loss + Virus + Phenotype + Academic_achievement + Longitudinal_study + Social_science + Economics + Carbon + Volcano + Perception + Oceanography + Ecosystem + Climate_change + Environmental_health + Laser + Radiation + Anxiety + Communication + Intensive_care_medicine + Big_Five_personality_traits + Trait + Single_nucleotide_polymorphism + Locus__genetics_ + Genome_wide_association_study + Innate_immune_system + Allele + Gene + Architecture + Computer_security + Geophysics + Geomorphology + Knowledge_management + Empathy + Pedagogy + Politics + Sociology + Emergency_medicine + Kidney_disease + Life_expectancy + Mortality_rate + Proportional_hazards_model + Medical_record + Gastroenterology + Radiation_therapy + Computer_vision + Artificial_intelligence + Advertising + Multimedia + Molecular_biology + Stem_cell + Cell_biology + Transcriptome + Cancer_cell + In_vitro + Incidence__epidemiology_ + Interquartile_range + General_surgery + Cognitive_psychology + Mood + Mutant + Atmospheric_sciences + Climate_model + Suicide_prevention + Injury_prevention + Human_factors_and_ergonomics + Occupational_safety_and_health + Microbiome + Habitat + Extinction + Threatened_species + Zoology + Insulin + Insulin_resistance + Adipose_tissue + Global_warming + Body_weight + Affect__psychology_ + Arousal + Environmental_engineering + Regimen + Geography + Demography + Offspring + Antigen + Receptor + Mutation + Mitochondrion + Astrobiology + Mineralogy + Photon + Botany + Intracellular + History + Demographics + Ethnic_group + Comorbidity + Programmed_cell_death + Downregulation_and_upregulation + Lung + Signal_transduction + Statistics + Mathematics + Nanoparticle + Quantum + Thin_film + Composite_material + Materials_science + Vitamin + Pathogenesis + Neuroscience + Chronic_pain + Evidence_based_medicine + Blood_pressure + Dopamine + Agroforestry + Hazard_ratio + Environmental_protection + Air_pollution + Cellular_differentiation + T_cell + Transplantation + Psychological_resilience + Inflammation + Kinase + Public_policy + Finance + Pediatrics + microRNA + Alzheimer_s_disease + Neuroimaging + Brain_mapping + Cardiology + Referral + Foraging + Demographic_economics + Labour_economics + Economic_growth + Social_perception + Social_cognition + Antibiotics + Polymer + Particle + Species_richness + Food_science + Autism + Vaccination + Geochemistry + Obstetrics + Obesity + Breastfeeding + Analytical_chemistry + Nanotechnology + Inorganic_chemistry + Addiction + Paleontology + Gut_flora + Antibody + Case_control_study + Gene_expression + Narrative + Electroencephalography + Social_environment + Interpersonal_communication + Bioinformatics + Biomarker__medicine_ + Sensory_system + Ion + Particle_physics + Remote_sensing + Actuarial_science + Coping__psychology_ + Bipolar_disorder + Catalysis + Functional_magnetic_resonance_imaging + Stimulus__physiology_ + Visual_perception + DNA_methylation + Medical_prescription + Regulation_of_gene_expression + RNA + Physical_medicine_and_rehabilitation + Happiness + Outbreak + Physiology + Spectroscopy + Nuclear_magnetic_resonance + Metabolic_syndrome + Relative_risk + Antimicrobial + Graphene + Mental_illness + Psychopathology + Membrane + Early_childhood + Transcription_factor + Embryonic_stem_cell + Ideology + Socioeconomics + Prostate_cancer + Survival_rate + Ingestion + Electron + Well_being + Weight_gain + Colorectal_cancer + Lung_cancer + Poverty + Greenhouse_gas + Waste_management + Coronary_artery_disease + Type_2_diabetes + Environmental_chemistry + Oncology + Physical_exercise + Incentive + Medicaid + Etiology + Schizophrenia + Heart_failure + The_Internet + Biotechnology + Breast_cancer + Clinical_endpoint + Chemotherapy + Radiology + Personality + Precipitation + Vulnerability + Epigenetics + Land_use + Ecosystem_services + Biological_dispersal + Testosterone + Veterinary_medicine + European_union + Gestation + Adverse_effect + Vitamin_D_and_neurology + Oxidative_stress + Gestational_age + Mesenchymal_stem_cell + Rehabilitation + Microeconomics + Hormone + Prefrontal_cortex + DNA + Genome + Heart_rate + Computer_science + Photochemistry + China + Public_administration + Health_policy + Feeling + Evolutionary_biology + Vegetation + Autism_spectrum_disorder + Audiology + MEDLINE + Toxicology + Fetus + Law + Social_media + Cause_of_death + Epilepsy + Agriculture + Bacteria + Cholesterol + Confounding + Human_brain + Fishery + Metal + Metabolism + Opioid + Antibiotic_resistance + Homeostasis + Metastasis + Malaria + Neurodegeneration + Immunotherapy + Severity_of_illness + Crossover_study + Child_development + Norm__social_ + Focus_group + Harm + Applied_psychology + Social_support + Emergency_department + Crystal + Amygdala + Empirical_research + Seismology + CRISPR + Nutrient + Guideline + Simulation + Alcohol + Distress + Archaeology + Molecular_physics + Cell_culture + Legislation + Health_equity + Gender_studies + Allergy + Developing_country + Cannabis + Meteorology + Myocardial_infarction + Progenitor_cell + DNA_sequencing + Hippocampal_formation + Hippocampus + Stimulation + Human_sexuality + Interpersonal_relationship + Working_memory + Immigration + Democracy + Electronic_engineering + Genotype + Predation + Major_depressive_disorder + Proinflammatory_cytokine + Developed_country + Phosphorylation + Phylogenetic_tree + National_Health_and_Nutrition_Examination_Survey + Data_mining + Text_mining + Circadian_rhythm + Arctic + Multiple_sclerosis + Chromatin + Induced_pluripotent_stem_cell + Transmission__mechanics_ + Machine_learning + Crystallography + Biomedical_engineering + Cell_type + Antioxidant + Fertility + Sediment + Criminology + Recall + Athletes + Atmosphere + Systematic_review + Instability + Genomics + Phylogenetics + Cytotoxic_T_cell + Computational_biology + Cell_growth + Carcinogenesis + Black_hole + Scattering + Genetic_variation + Prosocial_behavior"

# 199 keywords
keywords <- " + Cell_biology + Genetics + Biology + Body_mass_index + Health_care + Disease + Gerontology + Population + Public_health + Medicine + Materials_science + Composite_material + Nanotechnology + Cohort_study + Social_psychology + Cohort + Psychological_intervention + Young_adult + Family_medicine + Cancer + Surgery + Randomized_controlled_trial + Placebo + Clinical_trial + Nursing + Applied_psychology + Human_factors_and_ergonomics + Injury_prevention + Suicide_prevention + Psychiatry + Occupational_safety_and_health + Intensive_care_medicine + Pediatrics + Hazard_ratio + Confidence_interval + Retrospective_cohort_study + Vaccination + Psychology + Perception + Cognition + Environmental_health + Obesity + Risk_factor + Quality_of_life + Physical_therapy + Weight_loss + Anatomy + Mental_health + Psychosocial + Anxiety + Distress + Business + Public_relations + Marketing + Immunology + Global_warming + Economics + Climatology + Climate_change + General_surgery + Endocrinology + Internal_medicine + Receptor + Inflammation + Stimulus__physiology_ + Immune_system + Meta_analysis + Sociology + Gene + Cancer_research + Breast_cancer + Cell + Diabetes_mellitus + Blood_pressure + Oncology + Gynecology + Communication + Cognitive_psychology + Adverse_effect + Clinical_endpoint + Pharmacology + Virology + Risk_assessment + Transcription_factor + Political_science + Ecology + Geography + Cross_sectional_study + Odds_ratio + Comorbidity + Environmental_engineering + Chemistry + Medical_emergency + Physics + Social_science + Ethnic_group + Labour_economics + Antibody + Geomorphology + Geophysics + Geology + Ranging + Stroke + Environmental_resource_management + Type_2_diabetes + Cardiology + Molecular_biology + Developmental_psychology + Agriculture + Signal_transduction + Optoelectronics + Psychotherapist + Affect__psychology_ + Clinical_psychology + Anesthesia + Atmospheric_sciences + In_vivo + Biochemistry + Analytical_chemistry + Neuroscience + Botany + Gene_expression + Politics + Demography + Socioeconomic_status + Mortality_rate + Virus + Optics + Condensed_matter_physics + Bioinformatics + Law + Physical_medicine_and_rehabilitation + Stem_cell + Biodiversity + Astrophysics + Astronomy + Radiology + Pathology + Proportional_hazards_model + Chemotherapy + Predation + Food_science + Artificial_intelligence + Overweight + Antibiotics + Microbiology + Zoology + Paleontology + Habitat + Public_administration + Ecosystem + Economic_growth + Organic_chemistry + Government + Autism + Transplantation + Gastroenterology + Insulin + Engineering + Computer_science + Observational_study + Heart_disease + Epidemiology + Obstetrics + Pregnancy + Fishery + Alternative_medicine + Logistic_regression + Offspring + Mood + Bacteria + Prostate_cancer + Evolutionary_biology + Phenomenon + Longitudinal_study + Genome + Mutation + Pedagogy + Dementia + Relative_risk + Microeconomics + Odds + Feeling + Oceanography + Emergency_medicine + Personality + Prospective_cohort_study + Hippocampus + Greenhouse_gas + Biomarker__medicine_ + Myocardial_infarction + Socioeconomics + Drug + Environmental_science + Epigenetics + Inorganic_chemistry + Emergency_department + Medical_prescription + Phenotype"

In [32]:
equation_bar <- as.formula(paste(base_str, keywords, " + (1|journal_title) + (1|outlet)", sep = " "))
m5 <- glmer(formula = equation_bar, data = mydata, family = "binomial", control = glmerControl(optimizer = "nloptwrap"), nAGQ = 0)

Warning message:
“Some predictor variables are on very different scales: consider rescaling”

In [33]:
tidy(m5)

term,estimate,std.error,statistic,p.value,group
(Intercept),9.681466e-01,8.516415e-02,11.3680066,6.035011e-30,fixed
author_eth_ethnea_broadAfrican,-3.660354e-01,5.216946e-02,-7.0162765,2.278589e-12,fixed
author_eth_ethnea_broadChinese,-3.762245e-01,1.604767e-02,-23.4441761,1.515695e-121,fixed
author_eth_ethnea_broadEastAsian,-2.720950e-01,1.998486e-02,-13.6150563,3.258945e-42,fixed
author_eth_ethnea_broadEasternEuropean,-9.208412e-03,2.046940e-02,-0.4498624,6.528097e-01,fixed
author_eth_ethnea_broadIndian,-1.114116e-02,1.909284e-02,-0.5835254,5.595397e-01,fixed
author_eth_ethnea_broadMiddleEastern,1.559511e-02,1.724273e-02,0.9044455,3.657592e-01,fixed
author_eth_ethnea_broadSouthernEuropean,-1.377195e-01,1.308010e-02,-10.5289280,6.355412e-26,fixed
author_eth_ethnea_broadunknown,-2.270461e-01,5.298014e-02,-4.2854938,1.823338e-05,fixed
author_eth_ethnea_broadWesternNorthernEuropean,-7.227510e-02,1.026736e-02,-7.0393068,1.931985e-12,fixed


In [34]:
write.csv(tidy(m5), "/shared/0/projects/news-quotes/reg_results/reg_table/m5_coeff.csv", row.names = FALSE)

In [ ]:
# Difference between CIs in margins and marginal_effects: 
# https://github.com/leeper/margins/issues/135

margins

In [ ]:
margins_eff <- margins(m5, data = mydata, variables = c("author_eth_ethnea_broad", "author_gender_ethnea"))

In [ ]:
nrow(margins_eff)

In [ ]:
summary(margins_eff)

In [ ]:
write.csv(summary(margins_eff), "/shared/0/projects/news-quotes/reg_results/reg_table/big_mfx_table_R_margins.csv", row.names = FALSE)

## Only US institutions

In [31]:
subdata = mydata[mydata$affiliation_cate == 'domestic', ]

In [33]:
base_str_bar <- "is_author_mentioned ~ 1 + author_eth_ethnea_broad + author_gender_ethnea + reporter_eth_ethnea_broad + reporter_gender_ethnea + \
          last_name_length + last_name_prob + author_pos_cate + author_rank + is_top_author + is_corresponding + \
          affiliation_rank + num_authors + mention_year_center + gap_in_years + \
          num_words + num_mentioned_papers + FleschReadingEase + sentences_per_paragraph + type_token_ratio"

In [34]:
equation_bar <- as.formula(paste(base_str_bar, keywords, " + (1|journal_title) + (1|outlet)", sep = " "))
m5_us <- glmer(formula = equation_bar, data = subdata, family = "binomial", control = glmerControl(optimizer = "nloptwrap"), nAGQ = 0)

fixed-effect model matrix is rank deficient so dropping 5 columns / coefficients
Warning message:
“Some predictor variables are on very different scales: consider rescaling”

In [ ]:
# stargazer(m5_us, type = "latex", single.row=TRUE, ci = TRUE, report = ('vcsp'))

In [ ]:
# tidy(m5_us)

In [ ]:
write.csv(tidy(m5_us), "/shared/0/projects/news-quotes/reg_results/reg_table/m5_us_coeff.csv", row.names = FALSE)

In [28]:
margins_eff <- margins(m5_us, data = subdata, variables = c("author_eth_ethnea_broad", "author_gender_ethnea"))

In [29]:
nrow(margins_eff)

[1] 317626

In [ ]:
# summary(margins_eff)

In [31]:
write.csv(summary(margins_eff), "/shared/0/projects/news-quotes/reg_results/reg_table/big_mfx_table_R_margins_us_subset.csv", row.names = FALSE)


## non-US institutions

In [ ]:
nrow(mydata[mydata$affiliation_cate == 'international', ])

In [ ]:
base_str_bar <- "is_author_mentioned ~ 1 + author_eth_ethnea_broad + author_gender_ethnea + reporter_eth_ethnea_broad + reporter_gender_ethnea + \
          last_name_length + last_name_prob + author_pos_cate + author_rank + is_top_author + is_corresponding + \
          affiliation_rank + num_authors + mention_year_center + gap_in_years + \
          num_words + num_mentioned_papers + FleschReadingEase + sentences_per_paragraph + type_token_ratio"

In [ ]:
equation_bar <- as.formula(paste(base_str_bar, keywords, " + (1|journal_title) + (1|outlet)", sep = " "))
m5_intl <- glmer(formula = equation_bar, data = mydata[mydata$affiliation_cate == 'international', ], family = "binomial", control = glmerControl(optimizer = "nloptwrap"), nAGQ = 0)

In [ ]:
tidy(m5_intl)

In [ ]:
write.csv(tidy(m5_intl), "/shared/0/projects/news-quotes/reg_results/reg_table/m5_intl_coeff.csv", row.names = FALSE)